# Stacking and Blending (Assignment 4)

## Student: Rodolfo Lerma

In this lab assignment, you will build a non-tree-based classifier where you can ensemble any base-learners and pass it to the using a BaggingClassifier or any of the other ensemble learners in sklearn.ensemble.

In [1]:
# import packages
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import pandas as pd
from sklearn.datasets import make_moons
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample

# make this notebook's output stable across runs
np.random.seed(0)

## Data Set Information:

This dataset represents a set of possible advertisements on Internet pages. The features encode the geometry of the image (if available) as well as phrases occuring in the URL, the image's URL and alt text, the anchor text, and words occuring near the anchor text. The task is to predict whether an image is an advertisement ("ad") or not ("nonad"). Additional information can be found [here](https://archive.ics.uci.edu/ml/datasets/internet%2Badvertisements).

## Attribute Information:

The dataset has 3 continous (height, width, aratio) and 1555 binary (urls, tags, captions) features. 

## Source:

Creator & donor: Nicholas Kushmerick <nick '@' ucd.ie>

In [ ]:
# Load the data
internetAd = pd.read_csv('Internet_Ad_Data.csv', sep=',', error_bad_lines=False)
print(internetAd.info())
internetAd.head(20)

## Question 1: Prepare and impute missing values with the median

In [ ]:
#TO DO

## Question 2: Split dataset into training and test set

In [ ]:
from sklearn.model_selection import train_test_split

X = #TO DO
y = #TO DO

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



## Question 3: Train and evaluate a LogisticRegression classifier using LogisticRegression.Use Gridsearch CV to determine the best k.

In [ ]:
dtc_grid = #TO DO
dtc_grid.fit(#TO DO)

In [ ]:

# make predictions with the trained random forest
test_z = dtc_grid.predict(#TO DO)
test_z_prob = dtc_grid.predict_proba(#TO DO)

# evaluate the model performance - AUC and ROC

## Question 4: Use BaggingClassifier to train and evaluate an ensemble model of LogisticRegression  base classifiers. Each base classifier should be trained only on a sample half the size of the training data, and using only half as many features as there are in in total the training data (read the documentation for the function to see how to do this).

In [ ]:
bagOkNN = #TO DO)
bagOkNN.fit(#TO DO)

In [ ]:
# make predictions with the trained random forest
test_z = #TO DO
test_z_prob = #TO DO

# evaluate the model performance - AUC and ROC

## Question 5: Use AdaBoostClassifier to train and evaluate an ensemble model of LogisticRegression base classifiers.

In [ ]:
boostOkNN = #TO DO
boostOkNN.fit(#TO DO)

In [ ]:
# make predictions with the trained random forest
test_z = #TO DO
test_z_prob = #TO DO

# evaluate the model performance - AUC and ROC

## [Bonus] Question 6: Use StackingClassifier to train and evaluate an ensemble model of LogisticRegression base classifiers to get better accuracy than previous classifiers.

## Question 7: Create a new text cell in your Notebook: Complete a 50-100 word summary (or short description of your thinking in applying this week's learning to the solution) of your experience in this assignment. Include: 

- What was your incoming experience with this model, if any? 
- What steps you took, what obstacles you encountered? 
- How you link this exercise to real-world, machine learning problem-solving?
- What steps were missing? What else do you need to learn?